In [1]:
import pandas as pd
import numpy as np
import os
from analysis import *
import glob

In [115]:
models = [
    'Checkpoints_Edges_301_LR_0.13828312564892567_Head_CosFace_Optimizer_SGD_555',
#     'Checkpoints_Edges_600_LR_0.6708884458871945_Head_CosFace_Optimizer_SGD_555',
    'dpn107_CosFace_SGD_0.1_cosine',
    'rexnet_200_CosFace_SGD_0.1_cosine',
    'swin_base_patch4_window7_224_CosFace_SGD_0.1_cosine',
#     'inception_resnet_v2_CosFace_AdamW_0.001_cosine'
]
methods = [('baseline',''), 
          ('flipped','_finetune'), 
          ('angles','_finetune'), 
          ('discriminator','_finetune')]

dataset = 'test'
epochs = {'Checkpoints_Edges_301_LR_0.13828312564892567_Head_CosFace_Optimizer_SGD_555flipped_finetune': 8,
 'Checkpoints_Edges_301_LR_0.13828312564892567_Head_CosFace_Optimizer_SGD_555angles_finetune': 8,
 'Checkpoints_Edges_301_LR_0.13828312564892567_Head_CosFace_Optimizer_SGD_555discriminator_finetune': 10,
 'dpn107_CosFace_SGD_0.1_cosineflipped_finetune': 10,
 'dpn107_CosFace_SGD_0.1_cosineangles_finetune': 10,
 'dpn107_CosFace_SGD_0.1_cosinediscriminator_finetune': 9,
 'rexnet_200_CosFace_SGD_0.1_cosineflipped_finetune': 8,
 'rexnet_200_CosFace_SGD_0.1_cosineangles_finetune': 10,
 'rexnet_200_CosFace_SGD_0.1_cosinediscriminator_finetune': 4,
 'swin_base_patch4_window7_224_CosFace_SGD_0.1_cosineflipped_finetune': 11,
 'swin_base_patch4_window7_224_CosFace_SGD_0.1_cosineangles_finetune': 11,
 'swin_base_patch4_window7_224_CosFace_SGD_0.1_cosinediscriminator_finetune': 2}
metadata = pd.read_csv(f'vggface2_{dataset}_identities_gender.csv')



line = ['Model']
for method,s in methods:
    if method =='baseline' and s == '_finetune':
        continue
    line += [method+s]
print('  |  '.join(line))

for model in models:
    line = [model]
    for method, s in methods:
        if method == 'baseline':
            if model[0] == 'C':
                try:
                    df = pd.read_csv(f'vggface_phase1_{dataset}.csv', index_col='index')
                    line += [';'.join(["(%.2f" % ((df.loc[model[:-4]]['Accuracy_mean'])*100),
                                       "%.2f)" % df.loc[model[:-4]]['Disparity_mean']])]
                except:
                    line +=[',']
            else:
                df = pd.read_csv(f'vggface_phase1_{dataset}.csv', index_col='index')
                line += [';'.join(["(%.2f" % ((df.loc[model]['Accuracy_mean'])*100),
                                   "%.2f)" % df.loc[model]['Disparity_mean']])]
        else:        
            folder = f'vggface2_baselines/{model}_{method}{s}'
            files = glob.glob(folder+f'/*_{dataset}.pkl')
            if len(files):
                files.sort(key=lambda x: int(x.split('_')[-2]))
                last = files[-1]
                epoch = int(last.split('_')[-2])
                acc_df_vgg, acc_disp_df_vgg, rank_df_vgg = analyze_pickle_files([files], metadata)
                _, acc_disp_ratio_df_vgg, rank_ratio_df_vgg = analyze_pickle_files([files], metadata, ratio=True)
                err_df, error_ratio_df, _ = analyze_pickle_files([files], metadata, ratio=True, error=True)                
                acc_df_vgg = acc_df_vgg.dropna()
                
                df = error_ratio_df
                df = df.dropna()

                acc_df_vgg['Metric'] = 1 - acc_df_vgg['Metric']

                epoch = epochs[model+method+s]

                error = acc_df_vgg[acc_df_vgg['epoch'] == epoch]['Metric'].values[0]
                disparity = df[df['epoch'] == epoch]['Metric'].values[0]
                line += [';'.join(["(%.2f" % (error*100), 
                                   "%.2f)" % disparity])]
            else:
                line += ['']
    print('  |  '.join(line)+'')

Model  |  baseline  |  flipped_finetune  |  angles_finetune  |  discriminator_finetune
Checkpoints_Edges_301_LR_0.13828312564892567_Head_CosFace_Optimizer_SGD_555  |  (3.66;0.23)  |  (4.95;0.29)  |  (4.14;0.60)  |  (6.14;0.52)
dpn107_CosFace_SGD_0.1_cosine  |  (3.98;0.27)  |  (5.87;0.62)  |  (6.06;0.62)  |  (4.78;0.69)
rexnet_200_CosFace_SGD_0.1_cosine  |  (4.09;0.27)  |  (5.73;0.57)  |  (5.47;0.59)  |  (4.75;0.58)
swin_base_patch4_window7_224_CosFace_SGD_0.1_cosine  |  (5.47;0.38)  |  (5.75;0.56)  |  (5.23;0.60)  |  (5.03;0.60)


In [92]:
folder = f'vggface2_train*/{model}/'
files = glob.glob(folder+f'/*_{dataset}.pkl')
acc = []
disp = []
for f in files:
    acc_df_vgg, acc_disp_df_vgg, rank_df_vgg = analyze_pickle_files([[f]], metadata)
    acc.append(float(acc_df_vgg.dropna()['Metric']))
    disp.append(float(rank_df_vgg.dropna()['Metric']))
np.mean(acc), np.mean(disp)

In [41]:
models = [
    'Checkpoints_Edges_301_LR_0.13828312564892567_Head_CosFace_Optimizer_SGD_555',
#     'Checkpoints_Edges_600_LR_0.6708884458871945_Head_CosFace_Optimizer_SGD_555',
    'dpn107_CosFace_SGD_0.1_cosine',
    'rexnet_200_CosFace_SGD_0.1_cosine',
    'swin_base_patch4_window7_224_CosFace_SGD_0.1_cosine',
#     'inception_resnet_v2_CosFace_AdamW_0.001_cosine'
]
methods = ['baseline','flipped','angles','discriminator']
scratch = ['','_finetune']

dataset = 'test'

metadata = pd.read_csv(f'vggface2_{dataset}_identities_gender.csv')



line = ['Model']
for method in methods:
    for s in scratch:
        if method =='baseline' and s == '_finetune':
            continue
        line += [method+s]
print('  |  '.join(line))

for model in models:
    line = [model]
    for method, s in [('baseline',''), 
                      ('flipped','_finetune'), 
                      ('angles','_finetune'), 
                      ('discriminator','_finetune')]:
        if method == 'baseline':
            if model[0] == 'C':
                try:
                    df = pd.read_csv(f'vggface_phase1_{dataset}.csv', index_col='index')
                    line += [';'.join(["(%.2f" % ((df.loc[model[:-4]]['Accuracy_mean'])*100),
                                       "%.2f)" % df.loc[model[:-4]]['Disparity_mean']])]
                except:
                    line +=[',']
            else:
                df = pd.read_csv(f'vggface_phase1_{dataset}.csv', index_col='index')
                line += [';'.join(["(%.2f" % ((df.loc[model]['Accuracy_mean'])*100),
                                   "%.2f)" % df.loc[model]['Disparity_mean']])]
        else:        
            folder = f'vggface2_baselines/{model}_{method}{s}'
            files = glob.glob(folder+f'/*_{dataset}.pkl')
            if len(files):
                files.sort(key=lambda x: int(x.split('_')[-2]))
                last = files[-1]
                epoch = int(last.split('_')[-2])
                acc_df_vgg, acc_disp_df_vgg, rank_df_vgg = analyze_pickle_files([files], metadata)
                acc_df_vgg = acc_df_vgg.dropna()
                
                df = acc_disp_df_vgg
                df = df.dropna()

                acc_df_vgg['Metric'] = 1 - acc_df_vgg['Metric']

                epoch = epochs[model+method+s]

                error = acc_df_vgg[acc_df_vgg['epoch'] == epoch]['Metric'].values[0]
                disparity = df[df['epoch'] == epoch]['Metric'].values[0]
                line += [';'.join(["(%.2f" % (error*100), 
                                   "%.2f)" % disparity, str(epoch)])]
            else:
                line += ['']
    print('  |  '.join(line)+'\\\\')

Model  |  baseline  |  flipped  |  flipped_finetune  |  angles  |  angles_finetune  |  discriminator  |  discriminator_finetune
Checkpoints_Edges_301_LR_0.13828312564892567_Head_CosFace_Optimizer_SGD_555  |  (3.66;0.23)  |  (4.95;0.02);8  |  (4.14;0.04);8  |  (6.14;0.04);10\\
dpn107_CosFace_SGD_0.1_cosine  |  (3.98;0.27)  |  (5.87;0.05);10  |  (6.06;0.05);10  |  (4.78;0.05);9\\
rexnet_200_CosFace_SGD_0.1_cosine  |  (4.09;0.27)  |  (5.73;0.05);8  |  (5.47;0.05);10  |  (4.75;0.04);4\\
swin_base_patch4_window7_224_CosFace_SGD_0.1_cosine  |  (5.47;0.38)  |  (7.07;0.03);1  |  (5.23;0.04);11  |  (5.03;0.04);2\\


In [36]:
epochs

{'Checkpoints_Edges_301_LR_0.13828312564892567_Head_CosFace_Optimizer_SGD_555flipped': 8,
 'Checkpoints_Edges_301_LR_0.13828312564892567_Head_CosFace_Optimizer_SGD_555angles': 8,
 'Checkpoints_Edges_301_LR_0.13828312564892567_Head_CosFace_Optimizer_SGD_555discriminator': 10,
 'dpn107_CosFace_SGD_0.1_cosineflipped': 10,
 'dpn107_CosFace_SGD_0.1_cosineangles': 10,
 'dpn107_CosFace_SGD_0.1_cosinediscriminator': 9,
 'rexnet_200_CosFace_SGD_0.1_cosineflipped': 8,
 'rexnet_200_CosFace_SGD_0.1_cosineangles': 10,
 'rexnet_200_CosFace_SGD_0.1_cosinediscriminator': 4,
 'swin_base_patch4_window7_224_CosFace_SGD_0.1_cosineflipped': 1,
 'swin_base_patch4_window7_224_CosFace_SGD_0.1_cosineangles': 11,
 'swin_base_patch4_window7_224_CosFace_SGD_0.1_cosinediscriminator': 2}

Model&baseline&flipped&flipped_finetune&angles&angles_finetune&discriminator&discriminator_finetune
Checkpoints_Edges_301_LR_0.13828312564892567_Head_CosFace_Optimizer_SGD_555&(3.66;0.23)&7(5.83;0.39)&11(3.64;0.28)&5(6.16;0.42)\\
dpn107_CosFace_SGD_0.1_cosine&(3.98;0.27)&10(5.87;0.32)&10(6.06;0.36)&3(4.70;0.41)\\
rexnet_200_CosFace_SGD_0.1_cosine&(4.09;0.27)&11(5.63;0.50)&11(5.46;0.29)&6(4.90;0.35)\\
swin_base_patch4_window7_224_CosFace_SGD_0.1_cosine&(5.47;0.38)&11(5.75;0.44)&11(5.23;0.25)&4(5.03;0.30)\\
